## Processing QM9 dataset (134 k molecules) 
This notebook shows how to import the [QM9 dataset](https://figshare.com/articles/Data_for_6095_constitutional_isomers_of_C7H10O2/1057646) and generate featurizations using the MML toolkit, saving the feature vectors in .csv format. 

The filemformat from the paper
Raghunathan Ramakrishnan, Pavlo O. Dral, Matthias Rupp & O. Anatole von Lilienfeld. "[Quantum chemistry structures and properties of 134 kilo molecules](https://www.nature.com/articles/sdata201422) " *Scientific Data* **1**, 140022 (2014)


## Create file list 

In [162]:
import numpy as np
import os 

base_dir = '/home/delton/Downloads/134k_molecules_QM9/'
file_list = os.listdir(base_dir)
num_mols = len(file_list)
print(num_mols)

133885


# Create function to read the authors ".xyz" format

In [112]:
def read_xyz(file_name):
    with open(file_name, 'rb') as file:
        num_atoms = int(file.readline())
        properties = file.readline().split()[1:17]
        properties = [num.replace(b'*^', b'e') for num in properties] 
        properties = [float(prop) for prop in properties]
        atom_types = [0]*num_atoms
        coords = np.array(np.zeros([num_atoms,3]))
        for na in range(num_atoms):
            coord_line = file.readline().split()
            atom_types[na] = coord_line[0]
            xyz_coords = coord_line[1:4]
            xyz_coords = [num.replace(b'*^', b'e') for num in xyz_coords] 
            coords[na,:] = [float(num) for num in xyz_coords]  
        vib_freqs = file.readline()
        smiles = file.readline().split()[0]
        inchis = file.readline()
        
    return smiles, properties, atom_types, coords
        
    


## Read in data to lists

In [113]:
smiles = [0]*num_mols
properties = [0]*num_mols 
atom_types = [0]*num_mols
coords = [0]*num_mols

for im in range(num_mols):
    smiles[im], properties[im], atom_types[im], coords[im] = read_xyz(base_dir+file_list[im])

biggest_mol_size = max([len(atom_list) for atom_list in atom_types])

print("size of largest molecule = ", biggest_mol_size)    

size of largest molecule =  29


## Look at atom types

In [126]:
all_atoms = []
for atoms in atom_types:
    all_atoms += atoms
print(set(all_atoms))
del all_atoms

{b'O', b'N', b'H', b'F', b'C'}


## import mmltookit Coulomb matrix function and test

In [115]:
from mmltoolkit.featurizations import coulombmat_eigenvalues_from_coords

coulombmat_eigenvalues_from_coords(atom_types[1], coords[1], biggest_mol_size)


array([ 1.77473464e+02,  5.95943471e+01,  4.63087225e+01,  2.65512413e+01,
        2.29384657e+01,  8.57204543e+00,  7.22889442e+00,  2.50963775e+00,
       -1.10897971e+00, -7.92663621e-01, -5.70970218e-01, -4.21445985e-01,
       -2.83968522e-01, -6.65690196e-02, -6.55728683e-02,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00])

## generate Coulomb matrix eigenvalues

In [128]:
cmat_eigs = np.zeros([num_mols, biggest_mol_size])

for im in range(num_mols):
    cmat_eigs[im,:] = coulombmat_eigenvalues_from_coords(atom_types[im], coords[im], biggest_mol_size)


In [133]:
cmat_eigs.shape

(133885, 29)

## A single txt file that contains all the smiles strings.  


In [122]:
with open('QM9_all_smiles.txt', 'wb') as file:
    for smile in smiles:
        file.write(smile+b'\n')

## An excel file that contains the eignevalues of all Coulomb matrices.


In [139]:
np.savetxt('QM9_CM_eigenvalues.csv', cmat_eigs , delimiter=',')

## An excel file that contains all the eigenvalues of the weight matrices. (bond connections etc)


In [182]:
from rdkit.Chem.rdmolops import GetAdjacencyMatrix 

#----------------------------------------------------------------------------
def adjacency_matrix_eigenvalues(mol_list, useBO=False):

    eigenvalue_list = []
    max_length = 0

    for mol in mol_list:
        adj_matrix = GetAdjacencyMatrix(mol, useBO=useBO)
        evs = np.linalg.eigvals(adj_matrix)
        evs = sorted(evs, reverse=True) #sort
        evs = [float(ev) for ev in evs]
        eigenvalue_list += [evs]
        length = len(evs)
        if (length > max_length):
            max_length = length

    print(max_length)
    
    #zero padding
    for i in range(len(eigenvalue_list)):
        pad_width = max_length - len(eigenvalue_list[i])
        eigenvalue_list[i] += [0]*pad_width

    return np.array(eigenvalue_list)

In [190]:
#from mmltoolkit.featurizations import adjacency_matrix_eigenvalues
from rdkit import Chem

mol_list = [Chem.AddHs(Chem.MolFromSmiles(smile)) for smile in smiles]


In [191]:
adjmat_eigs = adjacency_matrix_eigenvalues(mol_list, useBO=True)

/home/delton/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: ComplexWarning: Casting complex values to real discards the imaginary part
  del sys.path[0]


In [192]:
adjmat_eigs.shape

(133885, 29)

In [194]:
np.savetxt('QM9_WAM_eigenvalues.csv', adjmat_eigs , delimiter=',')

## An excel file that contains the sum over bonds. 


In [195]:
from mmltoolkit.featurizations import sum_over_bonds

bond_types, X_LBoB  = sum_over_bonds(mol_list)

In [196]:
for bond_type in bond_types:
    print(bond_type+',', end='')


C-C,C-O,C-H,H-O,C:C,C=N,C:N,H-N,C-N,N:N,C=O,C#N,C#C,C=C,N:O,C:O,C-F,N-O,N=O,(133885, 19)


In [198]:
np.savetxt('QM9_SoB.csv', X_LBoB.astype('int') , fmt='%i', delimiter=',')


## An excel file that contains all the properties in the right order.

In [149]:
properties = np.array(properties)
properties.shape

(133885, 16)

In [152]:
np.savetxt('QM9_properties.csv', properties , delimiter=',')